In [4]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [6]:
class Sentimento(BaseModel):
    """Define o sentimento e a língua da mensagem enviada"""
    sentimento: str = Field(description="Sentimento do texto. Deve ser 'pos', 'neg' ou 'nd' para não definida.")
    lingua: str = Field(description="Língua que o texto foi escrito (deve estar no formato ISO 639-1)")
    
tool_sentimento = convert_to_openai_function(Sentimento)
tool_sentimento

{'name': 'Sentimento',
 'description': 'Define o sentimento e a língua da mensagem enviada',
 'parameters': {'type': 'object',
  'properties': {'sentimento': {'description': "Sentimento do texto. Deve ser 'pos', 'neg' ou 'nd' para não definida.",
    'type': 'string'},
   'lingua': {'description': 'Língua que o texto foi escrito (deve estar no formato ISO 639-1)',
    'type': 'string'}},
  'required': ['sentimento', 'lingua']}}

In [7]:
texto = "Eu gosto muito de massa aos quatro queijos"

In [12]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 

prompt = ChatPromptTemplate([
	("system", "Pense com cuidado ao categorizar o texto conforme as instrunções"),
	("user", "{input}")
])

chat = ChatOpenAI() 

chain = prompt | chat.bind(functions=[tool_sentimento], function_call={"name": "Sentimento"})

In [13]:
chain.invoke({"input":"Eu gosto muito de mass ao quatro queijos"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"pos","lingua":"pt"}', 'name': 'Sentimento'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 144, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0f4dbecb-61ba-435d-bd04-999d4cf5b4b4-0', usage_metadata={'input_tokens': 144, 'output_tokens': 11, 'total_tokens': 155})

In [14]:
duvidas = [
    "Bom dia, gostaria de saber se há um certificado final para cada trilha ou se os certificados são somente para os cursos e projetos? Obrigado!",
    "In Etsy, Amazon, eBay, Shopify https://pint77.com Pinterest+SEO +II = high sales results",
    "Boa tarde, estou iniciando hoje e estou perdido. Tenho vários objetivos. Não sei nada programação, exceto que utilizo o Power automate desktop da Microsoft. Quero aprender tudo na plataforma que se relacione ao Trading de criptomoedas. Quero automatizar Tradings, fazer o sistema reconhecer padrões, comprar e vender segundo critérios que eu defina, etc. Também tenho objetivos de aprender o máximo para utilizar em automações no trabalho também, que envolve a área jurídica e trabalho em processos. Como sou fã de eletrônica e tenho cursos na área, também queria aprender o que precisa para automatizacões diversas. Existe algum curso ou trilha que me prepare com base para todas essas áreas ao mesmo tempo e a partir dele eu aprenda isoladamente aquilo que seria exigido para aplicar aos meus projetos?",
    "Bom dia, Havia pedido cancelamento de minha mensalidade no mes 2 e continuaram cobrando. Peço cancelamento da assinatura. Peço por gentileza, para efetivarem o cancelamento da assomatura e pagamento.",
    "Bom dia. Não estou conseguindo tirar os certificados dos cursos que concluí. Por exemplo, já consegui 100% no python starter, porém, não consigo tirar o certificado. Como faço?",
    "Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?",
]

In [15]:
from enum import Enum 
from langchain.pydantic_v1 import BaseModel, Field 

class SetorEnum(str, Enum):
    atendimento_client = "atendimento_cliente"
    duvidas_aluno = "duvidas_alunos"
    vendas="vendas"
    spam="spam"
    
class DirecionaSetorResponsavel(BaseModel):
    """Direcione a dúvida de um cliente ou aluno da escola de programação asimov para o setor responsável"""
    setor: SetorEnum

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_function 

tool_direcionamento = convert_to_openai_function(DirecionaSetorResponsavel)
tool_direcionamento

{'name': 'DirecionaSetorResponsavel',
 'description': 'Direcione a dúvida de um cliente ou aluno da escola de programação asimov para o setor responsável',
 'parameters': {'type': 'object',
  'properties': {'setor': {'description': 'An enumeration.',
    'enum': ['atendimento_cliente', 'duvidas_alunos', 'vendas', 'spam'],
    'type': 'string'}},
  'required': ['setor']}}

In [17]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

prompt = ChatPromptTemplate.from_messages([
	("system", "Pense com cuidado ao categorizar o texto conforme as instruções"),
	('user', "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento], function_call={"name":"DirecionaSetorResponsavel"}) | JsonOutputFunctionsParser()) 



In [18]:
duvida = duvidas[5]

res = chain.invoke({"input": duvida})
print("Dúvida: ", duvida)
print("Resposta: ", res)

Dúvida:  Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?
Resposta:  {'setor': 'atendimento_cliente'}


In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a dúvidas de preço, sobre o produto, como funciona devem ser direciodas para "vendas".
Questões relacionadas a conta, acesso a plataforma, a cancelamento e renovação de assinatura para devem ser direciodas para "atendimento_cliente".
Questões relacionadas a dúvidas técnicas de programação, conteúdos da plataforma ou tecnologias na área da programação devem ser direciodas para "duvidas_alunos".
Mensagens suspeitas, em outras línguas que não português, contendo links devem ser direciodas para "spam".
'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = (prompt 
         | chat.bind(functions=[tool_direcionamento], function_call={'name': 'DirecionaSetorResponsavel'})
         | JsonOutputFunctionsParser())

In [20]:
for duvida in duvidas:
    res = chain.invoke({"input": duvida})
    print("Dúvida: ", duvida)
    print("Resposta: ", res)
    print("\n\n")
    

Dúvida:  Bom dia, gostaria de saber se há um certificado final para cada trilha ou se os certificados são somente para os cursos e projetos? Obrigado!
Resposta:  {'setor': 'duvidas_alunos'}



Dúvida:  In Etsy, Amazon, eBay, Shopify https://pint77.com Pinterest+SEO +II = high sales results
Resposta:  {'setor': 'spam'}



Dúvida:  Boa tarde, estou iniciando hoje e estou perdido. Tenho vários objetivos. Não sei nada programação, exceto que utilizo o Power automate desktop da Microsoft. Quero aprender tudo na plataforma que se relacione ao Trading de criptomoedas. Quero automatizar Tradings, fazer o sistema reconhecer padrões, comprar e vender segundo critérios que eu defina, etc. Também tenho objetivos de aprender o máximo para utilizar em automações no trabalho também, que envolve a área jurídica e trabalho em processos. Como sou fã de eletrônica e tenho cursos na área, também queria aprender o que precisa para automatizacões diversas. Existe algum curso ou trilha que me prepare com bas